__Import the functions__ (assumes that `QTLight_functions.py` is in your current working directory or in your python path)

In [2]:
import QTLight_functions as QTL

__Fetch relevant files from stacks populations run__

In [2]:
%%bash

#vcf result from populations
ln -s /media/chrishah/STORAGE/RAD/stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/3-populations/Diplotaxodon_4pop/m5_mpop5_kernel_iterate_ONE_SNP_PER_TAG/r_0.8-p_4/batch_1.vcf.gz .
#populationmap as used by stacks
ln -s /media/chrishah/STORAGE/RAD/stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/3-populations/Diplotaxodon_4pop/populationmap .
mkdir matrix

__create 10 Bayenv input files with 5000 randomly selected loci in each__

In [3]:
%%bash
#pip install pyvcf

for a in {1..10}
do
    echo -e "\nrepitition $a:\n"
    python /media/chrishah/STORAGE/Dropbox/Github/genomisc/popogeno/QTlight/vcf_2_div.py --min_number 5 -r 5000 -o matrix/random_5000_rep_$a -m populationmap --exclude_singletons batch_1.vcf.gz
done


repitition 1:


individuals per population (as specified by populationmap):
Di_1 (10): sample_BBNB1.1, sample_BBNB10.1, sample_BBNB2.1, sample_BBNB3.1, sample_BBNB4.1, sample_BBNB5.1, sample_BBNB6.1, sample_BBNB7.1, sample_BBNB8.1, sample_BBNB9.1
Di_2 (6): sample_LBPNK2.1, sample_LBPNK3.1, sample_LBPNK4.1, sample_LBPNK5.1, sample_LBPNK6.1, sample_LBPNK7.1
Di_4 (8): sample_MONB11.1, sample_MONB2.1, sample_MONB3.1, sample_MONB4.1, sample_MONB5.1, sample_MONB7.1, sample_MONB8.1, sample_MONB9.1
Di_5 (10): sample_NJ1.1, sample_NJ10.1, sample_NJ2.1, sample_NJ3.1, sample_NJ4.1, sample_NJ5.1, sample_NJ6.1, sample_NJ7.1, sample_NJ8.1, sample_NJ9.1

specified minimum counts of required individuals per population:
Di_1:	5
Di_2:	5
Di_4:	5
Di_5:	5

you specified minimum minor allele count: 2


requested randomly selected subset of 5000 SNPs

writing files:

	matrix/random_5000_rep_1.bayenv.SNPfile
	matrix/random_5000_rep_1.bayenv.SNPmap


repitition 2:


individuals per population (as specified by

__create 10 covariance matrizes with 100000 iterations each__

In [14]:
%%bash
cd matrix/

for a in {1..10}
do
    rand=$RANDOM
    echo -e "repitition $a (random seed: -$rand)\n"
    /home/chrishah/src/Bayenv/bayenv2 0 -p 4 -r -$RANDOM -k 100000 -i random_5000_rep_$a.bayenv.SNPfile > random_5000_rep_$a.log
    
done
cd ../

repitition 1 (random seed: -20158)

repitition 2 (random seed: -16468)

repitition 3 (random seed: -20371)

repitition 4 (random seed: -13871)

repitition 5 (random seed: -19331)

repitition 6 (random seed: -30726)

repitition 7 (random seed: -1792)

repitition 8 (random seed: -17363)

repitition 9 (random seed: -26042)

repitition 10 (random seed: -20693)



__extract covariance matrizes__ from final iteration into txt file

In [15]:
%%bash
dimensions=4

for a in {1..10}
do
    tail -n $((dimensions+1)) matrix/random_5000_rep_$a.log | grep "^$" -v > matrix/random_5000_rep_$a\_it-10e5.matrix
done


__construct average covariance matrix from 10 random sets__

In [16]:
import numpy as np
main_list = []

for a in range(10):
    current = "matrix/random_5000_rep_"+str(a+1)+"_it-10e5.matrix"
#    print current
    IN = open(current,"r")
    temp_list = []
    for line in IN:
        temp_list.extend(line.rstrip().split("\t"))

    for i in range(len(temp_list)):
        if a == 0:
            main_list.append([float(temp_list[i])])
        else:
            main_list[i].append(float(temp_list[i]))
        
#print main_list

av_out_list = []
std_out_list = []
for j in range(len(main_list)):
    av_out_list.append(np.mean(main_list[j]))

#print av_out_list

outstring = ""
for z in range(len(av_out_list)):
    av_out_list[z] = "%s\t" %av_out_list[z]
    if not outstring:
        outstring = av_out_list[z]
    else:
        outstring = outstring+av_out_list[z]
        if ((z+1) % 4 == 0):
            outstring = "%s\n" %(outstring)


OUT = open("matrix/av_matrix.matrix","w")
OUT.write(outstring)
OUT.close()
        

__Prepare__ environmental data - average and normalize

raw data is provided in a csv file with the first column containing the population id. See example in test-data.

In [24]:
%%bash
mkdir morpho_data
cat /media/chrishah/STORAGE/Dropbox/Diplotaxodon/Morpho_raw_data/Diplotaxodon_Morphometric_Data_corrected_TL.csv | sed 's/ /_/g' >  morpho_data/Diplotaxodon_Morphometric_Data_corrected_TL.csv


In [2]:
populations, IDs = QTL.normalize(csv='./morpho_data/Diplotaxodon_Morphometric_Data_corrected_TL.csv', normalize=True, norm_prefix='morpho_data/Diplotaxodon_Morphometric_Data_normalized', boxplot=True, boxplots_prefix="morpho_data/boxplot")

Creating boxplot for PC1

Creating boxplot for PC2

Creating boxplot for PC3

Creating boxplot for PC4

Creating boxplot for Head_Length

Creating boxplot for Body_Depth

Creating boxplot for CP_Length

Creating boxplot for CP_Depth

Creating boxplot for Snout_L

Creating boxplot for Snout_Dorsal

Creating boxplot for Snout_Pelvic

Creating boxplot for Pectoral_FL

Creating boxplot for Pelvic_FL

Creating boxplot for Dorsal_BL

Creating boxplot for Anal_BL

Creating boxplot for H_Ey_D

Creating boxplot for H_Ey_D_p_TL

Creating boxplot for V_Ey_D

Creating boxplot for V_EY_D_p_TL

Creating boxplot for POHL

Creating boxplot for POD

Creating boxplot for LJL

Creating boxplot for IOW

Creating boxplot for POW

Creating boxplot for CD

Creating boxplot for HD


normalizing 26 environmental factors across 4 populations
writing to:
	morpho_data/Diplotaxodon_Morphometric_Data_normalized.bayenv
	morpho_data/Diplotaxodon_Morphometric_Data_normalized.csv


In [3]:
print populations
print IDs

['Di_1', 'Di_2', 'Di_4', 'Di_5']
['Anal_BL', 'Body_Depth', 'CD', 'CP_Depth', 'CP_Length', 'Dorsal_BL', 'HD', 'H_Ey_D', 'H_Ey_D_p_TL', 'Head_Length', 'IOW', 'LJL', 'PC1', 'PC2', 'PC3', 'PC4', 'POD', 'POHL', 'POW', 'Pectoral_FL', 'Pelvic_FL', 'Snout_Dorsal', 'Snout_L', 'Snout_Pelvic', 'V_EY_D_p_TL', 'V_Ey_D']


__convert vcf to bayenv - generate full SNP file (excluding singletons)__

In [13]:
%%bash

mkdir SNPfiles
python /media/chrishah/STORAGE/Dropbox/Github/genomisc/popogeno/QTlight/vcf_2_div.py batch_1.vcf.gz --min_number 5 -o SNPfiles/full_set -m populationmap --exclude_singletons



individuals per population (as specified by populationmap):
Di_1 (10): sample_BBNB1.1, sample_BBNB10.1, sample_BBNB2.1, sample_BBNB3.1, sample_BBNB4.1, sample_BBNB5.1, sample_BBNB6.1, sample_BBNB7.1, sample_BBNB8.1, sample_BBNB9.1
Di_2 (6): sample_LBPNK2.1, sample_LBPNK3.1, sample_LBPNK4.1, sample_LBPNK5.1, sample_LBPNK6.1, sample_LBPNK7.1
Di_4 (8): sample_MONB11.1, sample_MONB2.1, sample_MONB3.1, sample_MONB4.1, sample_MONB5.1, sample_MONB7.1, sample_MONB8.1, sample_MONB9.1
Di_5 (10): sample_NJ1.1, sample_NJ10.1, sample_NJ2.1, sample_NJ3.1, sample_NJ4.1, sample_NJ5.1, sample_NJ6.1, sample_NJ7.1, sample_NJ8.1, sample_NJ9.1

specified minimum counts of required individuals per population:
Di_1:	5
Di_2:	5
Di_4:	5
Di_5:	5

you specified minimum minor allele count: 2


requested full set of 11786 SNPs (12262 loci were removed because they did not meet the minimum criteria)

writing files:

	SNPfiles/full_set.bayenv.SNPfile
	SNPfiles/full_set.bayenv.SNPmap



__split up SNPfiles into single files__

In [22]:
QTL.split_for_Bayenv(infile='SNPfiles/full_set.bayenv.SNPfile', out_prefix='SNPfiles/Diplo_SNP')

__Run Bayenv for 10 replications__ serially

In [23]:
#find the number of SNP files to add to specify in loop below
!ls -1 SNPfiles/Diplo* |wc -l

11786


In [20]:
!mkdir running_Bayenv

for this run I used bayenv2 version: `tguenther-bayenv2_public-48f0b51ced16`

In [ ]:
%%bash
#adjust bayenv command to your requirements


iterations=1000000

cd running_Bayenv/

for rep in {1..20}; do ran=$RANDOM; for a in {0000001..0023571}; do /home/chrishah/src/Bayenv/bayenv2_2801205/tguenther-bayenv2_public-48f0b51ced16/bayenv2 -i ../SNPfiles/Diplo_SNP-$a.txt -e ../morpho_data/Diplotaxodon_Morphometric_Data_normalized.bayenv -m ../matrix/av_matrix.matrix -k $iterations -r -$ran -p 4 -n 20 -t -X -c -o bayenv_out_k100000_env_rep_$rep-rand_$ran; done > log_rep_$rep; done

__ALTERNATIVE__

__Bayenv__ can be run on a HPC cluster in parallel. I provide a script `submit_Bayenv_array_multi.sh` that I used to run 10 replicates as arrayjob on a cluster that was running a PBS scheduling system. Total runtime for 10 replicates with 1M Bayenv iterations/SNP was ~ 24h. The results from the individual runs were then concatenated with the script `concat_sorted.sh` and moved to the directory `running_Bayenv` on the local machine.

#ANALYSE RANK STATISTICS#
please make sure you load all functions below first

__Calculating RANK STATISTICS__

In [5]:
mkdir RANK_STATISTIC_2/

In [4]:
#create the list of Bayenv results files to be processed
import os

bayenv_res_dir = './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates/'
bayenv_files = []

for fil in os.listdir(bayenv_res_dir):
    if fil.endswith(".bf"):
        print(bayenv_res_dir+"/"+fil)
        bayenv_files.append(bayenv_res_dir+"/"+fil)

./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_2.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_14.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_1.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_11.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_20.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_7.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_6.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_18.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_8.bf
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep

In [5]:
print bayenv_files
print "\n%i" %len(bayenv_files)
print IDs
print len(IDs)

['./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_2.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_14.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_1.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_11.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_20.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_7.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_6.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_18.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_8.bf', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singleton

In [6]:

rank_results = QTL.calculate_rank_stats(SNP_map="SNPfiles/full_set.bayenv.SNPmap", infiles = bayenv_files, ids = IDs, prefix = 'RANK_STATISTIC_2/Diplo_1M')

Total number of SNPs (according to the SNPmap): 11786
Number of Bayenv replicates: 20
Number of environmental factors analysed: 26
parsing bayenv files

processing replicate 0:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_1.bf

processing replicate 1:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.bf

processing replicate 2:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_11.bf

processing replicate 3:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_12.bf

processing replicate 4:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_13.bf

processing replicate 5:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_14.bf

processing replicate 6:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_15.bf

processing replicate 7:
./run

__CREATE POPE PLOTS and extract the SNP ids in the top 5 percent__ (assumes that the script `pope_plot.sh` is in your working directory)

In [11]:
!ln -s /media/chrishah/STORAGE/Dropbox/Github/genomisc/popogeno/QTlight/plot_pope.sh .

In [7]:
print IDs

['Anal_BL', 'Body_Depth', 'CD', 'CP_Depth', 'CP_Length', 'Dorsal_BL', 'HD', 'H_Ey_D', 'H_Ey_D_p_TL', 'Head_Length', 'IOW', 'LJL', 'PC1', 'PC2', 'PC3', 'PC4', 'POD', 'POHL', 'POW', 'Pectoral_FL', 'Pelvic_FL', 'Snout_Dorsal', 'Snout_L', 'Snout_Pelvic', 'V_EY_D_p_TL', 'V_Ey_D']


In [8]:
full_rank_files = []

file_dir = 'RANK_STATISTIC_2/'
for id in IDs:
#    print id
    for file in os.listdir(file_dir):
        if file.endswith('_'+id+'.txt'):
#            print [id,file_dir+'/'+file]
            full_rank_files.append([id,file_dir+'/'+file])
            break
        

In [9]:
print full_rank_files

[['Anal_BL', 'RANK_STATISTIC_2//Diplo_1M_Anal_BL.txt'], ['Body_Depth', 'RANK_STATISTIC_2//Diplo_1M_Body_Depth.txt'], ['CD', 'RANK_STATISTIC_2//Diplo_1M_CD.txt'], ['CP_Depth', 'RANK_STATISTIC_2//Diplo_1M_CP_Depth.txt'], ['CP_Length', 'RANK_STATISTIC_2//Diplo_1M_CP_Length.txt'], ['Dorsal_BL', 'RANK_STATISTIC_2//Diplo_1M_Dorsal_BL.txt'], ['HD', 'RANK_STATISTIC_2//Diplo_1M_HD.txt'], ['H_Ey_D', 'RANK_STATISTIC_2//Diplo_1M_H_Ey_D.txt'], ['H_Ey_D_p_TL', 'RANK_STATISTIC_2//Diplo_1M_smoothed_H_Ey_D_p_TL.txt'], ['Head_Length', 'RANK_STATISTIC_2//Diplo_1M_Head_Length.txt'], ['IOW', 'RANK_STATISTIC_2//Diplo_1M_IOW.txt'], ['LJL', 'RANK_STATISTIC_2//Diplo_1M_LJL.txt'], ['PC1', 'RANK_STATISTIC_2//Diplo_1M_PC1.txt'], ['PC2', 'RANK_STATISTIC_2//Diplo_1M_PC2.txt'], ['PC3', 'RANK_STATISTIC_2//Diplo_1M_PC3.txt'], ['PC4', 'RANK_STATISTIC_2//Diplo_1M_PC4.txt'], ['POD', 'RANK_STATISTIC_2//Diplo_1M_POD.txt'], ['POHL', 'RANK_STATISTIC_2//Diplo_1M_POHL.txt'], ['POW', 'RANK_STATISTIC_2//Diplo_1M_POW.txt'], ['Pec

In [10]:
QTL.plot_pope(files_list=full_rank_files, cutoff=0.95, num_replicates=20)

processing Anal_BL:
data in file: RANK_STATISTIC_2//Diplo_1M_Anal_BL.txt
RANK_STATISTIC_2//Diplo_1M_Anal_BL.txt
0.95
Anal_BL
RANK_STATISTIC_2//Diplo_1M_Anal_BL
20
null device 
          1 

processing Body_Depth:
data in file: RANK_STATISTIC_2//Diplo_1M_Body_Depth.txt
RANK_STATISTIC_2//Diplo_1M_Body_Depth.txt
0.95
Body_Depth
RANK_STATISTIC_2//Diplo_1M_Body_Depth
20
null device 
          1 

processing CD:
data in file: RANK_STATISTIC_2//Diplo_1M_CD.txt
RANK_STATISTIC_2//Diplo_1M_CD.txt
0.95
CD
RANK_STATISTIC_2//Diplo_1M_CD
20
null device 
          1 

processing CP_Depth:
data in file: RANK_STATISTIC_2//Diplo_1M_CP_Depth.txt
RANK_STATISTIC_2//Diplo_1M_CP_Depth.txt
0.95
CP_Depth
RANK_STATISTIC_2//Diplo_1M_CP_Depth
20
null device 
          1 

processing CP_Length:
data in file: RANK_STATISTIC_2//Diplo_1M_CP_Length.txt
RANK_STATISTIC_2//Diplo_1M_CP_Length.txt
0.95
CP_Length
RANK_STATISTIC_2//Diplo_1M_CP_Length
20
null device 
          1 

processing Dorsal_BL:
data in file: RANK_STAT

In [14]:
#calculate p values for H_Ey_D_p_TL
#identifying scaffolds that contain SNPS with ARR >0.95
IN = open('RANK_STATISTIC_2/Diplo_1M_H_Ey_D_p_TL-top-0.95.tsv')
scaffolds = []

for line in IN:
    if not line.startswith('chrom'):
            scaffolds.append(line.split("\t")[0])
IN.close()           
scaffolds = sorted(list(set(scaffolds)))

print "Scaffols to process:\n"+str(scaffolds)

QTL.smoothing(dictionary = rank_results, window_size=50000, bootstrap_reps=10000, factor='H_Ey_D_p_TL', 
              to_smooth='avg_rank_rel', verbose=1, chromos=scaffolds)

QTL.write_stats(dictionary=rank_results, 
            out_columns=['std_rank','avg_rank_rel','avg_rank_rel_smoothed','avg_rank_rel_smoothed_p'], 
            factor='H_Ey_D_p_TL', 
            chromos=scaffolds,
            prefix = './RANK_STATISTIC_2/Diplo_1M_smoothed')

Scaffols to process:
['scaffold_110', 'scaffold_12', 'scaffold_13', 'scaffold_145', 'scaffold_148', 'scaffold_149', 'scaffold_159', 'scaffold_16', 'scaffold_162', 'scaffold_19', 'scaffold_193', 'scaffold_197', 'scaffold_2', 'scaffold_21', 'scaffold_215', 'scaffold_23', 'scaffold_230', 'scaffold_236', 'scaffold_254', 'scaffold_258', 'scaffold_267', 'scaffold_269', 'scaffold_292', 'scaffold_30', 'scaffold_36', 'scaffold_361', 'scaffold_38', 'scaffold_39', 'scaffold_404', 'scaffold_44', 'scaffold_62', 'scaffold_74', 'scaffold_91']

### Kernel smoothing ###
Factor: H_Ey_D_p_TL
Statistic: avg_rank_rel
Window size: 50000 bp
determining p-value based on 10000 bootstrap replicates
processing scaffold_110
scaffold_110	274029	0.740595879156 - p = 0.049
scaffold_110	284607	0.771057634312 - p = 0.0259
scaffold_110	452484	0.85875931522 - p = 0.0005
scaffold_110	452630	0.863766927999 - p = 0.0004
scaffold_110	512023	0.802290941518 - p = 0.0005
scaffold_110	566581	0.799506298351 - p = 0.0002
scaffold

In [ ]:
#calculate p values for V_EY_D_p_TL
#identifying scaffolds that contain SNPS with ARR >0.95
IN = open('RANK_STATISTIC_2/Diplo_1M_V_EY_D_p_TL-top-0.95.tsv')
scaffolds = []

for line in IN:
    if not line.startswith('chrom'):
            scaffolds.append(line.split("\t")[0])
IN.close()           
scaffolds = sorted(list(set(scaffolds)))

print "Scaffols to process:\n"+str(scaffolds)

QTL.smoothing(dictionary = rank_results, window_size=50000, bootstrap_reps=10000, factor='V_EY_D_p_TL', 
              to_smooth='avg_rank_rel', verbose=1, chromos=scaffolds)

QTL.write_stats(dictionary=rank_results, 
            out_columns=['std_rank','avg_rank_rel','avg_rank_rel_smoothed','avg_rank_rel_smoothed_p'], 
            factor='V_EY_D_p_TL', 
            chromos=scaffolds,
            prefix = './RANK_STATISTIC_2/Diplo_1M_smoothed')

In [11]:
print rank_results['global']['H_Ey_D_p_TL'][1].keys()



['avg_rank_rel', 'p20_rank_rel', 'avg_rank', 'med_rank_rel', 'ranks', 'var_rank_rel', 'var_weighted_avg_rank', 'p20_rank', 'var_rank_weight', 'var_weighted_rel_avg_rank', 'med_rank_rel_smoothed', 'med_rank', 'std_rank', 'avg_rank_rel_smoothed_p', 'avg_rank_rel_smoothed', 'med_rank_rel_smoothed_p', 'mad_rank', 'var_rank']


__Apply stringent filters to identify candidate regions__

Filters:
 - ARR >= 0.95
 - smotthed ARR p < 0.001
 
 

In [44]:
!cat RANK_STATISTIC_2/Diplo_1M_smoothed_H_Ey_D_p_TL.txt RANK_STATISTIC_2/Diplo_1M_smoothed_V_EY_D_p_TL.txt | perl -ne 'chomp; @a=split("\t"); if (($a[-1] < 0.001) and ($a[-3] >= 0.95)){ print "\n$_\n"}' | cut -f 1-3 |sort | uniq > RANK_STATISTIC_2/EYE_regions.tsv 

In [51]:
!cat RANK_STATISTIC_2/EYE_regions.tsv


scaffold_12	3799847	6728
scaffold_148	543741	9559
scaffold_159	594470	11119
scaffold_197	125383	16065
scaffold_197	143069	16067
scaffold_197	150472	16069
scaffold_215	576905	19268
scaffold_39	1771937	31651


__find genes__ up and downstream of correlated SNPs

In [8]:
#make list desired rank statistic tsv files

regions = ['RANK_STATISTIC_2/EYE_regions.tsv']

__parse a gff__ file

In [9]:
gff_per_scaffold = QTL.parse_gff(gff='/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/annotations/Metriaclima_zebra.BROADMZ2.gtf')

__identify genes__ within a defined distance (in kb) up and down-stream of the SNPs

In [17]:
genes_per_analysis = QTL.find_genes(rank_stats = regions, gff = gff_per_scaffold, distance = 50)

processing rank statistic file: RANK_STATISTIC_2/EYE_regions.tsv
EYE_regions:
identified 71 gene(s)


__annotated relevant genes__ based on blast2go annotation table

In [18]:
QTL.annotate_genes(SNPs_to_genes=genes_per_analysis, annotations='/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/annotations/blast2GO/blast2GO/blast2go_table_20150630_0957.txt')

EYE_regions
adding annoation for EYE_regions


In [19]:
mkdir find_genes

mkdir: cannot create directory ‘find_genes’: File exists


__write__ summary table for SNPs and relevant genes in the vicinity

In [20]:
QTL.write_candidates(SNPs_to_genes=genes_per_analysis, whitelist=genes_per_analysis.keys(), out_dir='./find_genes/')

EYE_regions
writing to: ./find_genes/EYE_regions.genes.annotated.tsv


__Prepare list of genes for Enrichment analysis in BLAST2GO__ from most stringently filtered SNPs.

In [25]:
!cat find_genes/EYE_regions.genes.annotated.tsv | cut -f 4 | grep "gene" -v | sort -n | uniq > \
find_genes/EYE_regions.genes.txt

##Prepare less stringent test datasets for Enrichment analyses

Extract genes for loci with 'smoothed ARR p<0.05'.

In [12]:
%%bash

cat RANK_STATISTIC_2/Diplo_1M_smoothed_H_Ey_D_p_TL.txt RANK_STATISTIC_2/Diplo_1M_smoothed_V_EY_D_p_TL.txt | \
perl -ne 'chomp; @a=split("\t"); if ($a[-1] < 0.05){ print "\n$_\n"}' | grep "chrom" -v | cut -f 1-3 |sort | uniq \
> RANK_STATISTIC_2/EYE_regions.p-0.05.tsv  

In [13]:
#make list desired rank statistic tsv files

regions = ['RANK_STATISTIC_2/EYE_regions.p-0.05.tsv']

In [14]:
gff_per_scaffold = QTL.parse_gff(gff='/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/annotations/Metriaclima_zebra.BROADMZ2.gtf')

In [15]:
genes_per_analysis = QTL.find_genes(rank_stats = regions, gff = gff_per_scaffold, distance = 50)

processing rank statistic file: RANK_STATISTIC_2/EYE_regions.p-0.05.tsv
EYE_regions.p-0.05:
identified 743 gene(s)


In [16]:
QTL.annotate_genes(SNPs_to_genes=genes_per_analysis, annotations='/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/annotations/blast2GO/blast2GO/blast2go_table_20150630_0957.txt')

EYE_regions.p-0.05
adding annoation for EYE_regions.p-0.05


In [17]:
QTL.write_candidates(SNPs_to_genes=genes_per_analysis, whitelist=genes_per_analysis.keys(), out_dir='./find_genes/')

EYE_regions.p-0.05
writing to: ./find_genes/EYE_regions.p-0.05.genes.annotated.tsv


__Prepare list of genes for Enrichment analysis in BLAST2GO__ from SNPs with 'smoothed ARR p <0.05'

In [29]:
%%bash

cat find_genes/EYE_regions.p-0.05.genes.annotated.tsv | cut -f 4 | grep "gene" -v | sort | uniq \
> find_genes/EYE_regions.p-0.05.genes.txt

wc -l find_genes/EYE_regions.p-0.05.genes.txt

434 find_genes/EYE_regions.p-0.05.genes.txt


Extract genes for loci with ARR >= 0.95.

In [19]:
%%bash

cat RANK_STATISTIC_2/Diplo_1M_smoothed_H_Ey_D_p_TL.txt RANK_STATISTIC_2/Diplo_1M_smoothed_V_EY_D_p_TL.txt | \
perl -ne 'chomp; @a=split("\t"); if ($a[-3] >= 0.95){ print "\n$_\n"}' | grep "chrom" -v | cut -f 1-3 |sort | uniq \
> RANK_STATISTIC_2/EYE_regions.top-0.95.tsv

In [20]:
#make list desired rank statistic tsv files

regions = ['RANK_STATISTIC_2/EYE_regions.top-0.95.tsv']

In [21]:
genes_per_analysis = QTL.find_genes(rank_stats = regions, gff = gff_per_scaffold, distance = 50)

processing rank statistic file: RANK_STATISTIC_2/EYE_regions.top-0.95.tsv
EYE_regions.top-0.95:
identified 277 gene(s)


In [22]:
QTL.annotate_genes(SNPs_to_genes=genes_per_analysis, annotations='/media/chrishah/STORAGE/DATA/Cichlids/reference_data/M_zebra/annotations/blast2GO/blast2GO/blast2go_table_20150630_0957.txt')

EYE_regions.top-0.95
adding annoation for EYE_regions.top-0.95


In [23]:
QTL.write_candidates(SNPs_to_genes=genes_per_analysis, whitelist=genes_per_analysis.keys(), out_dir='./find_genes/')

EYE_regions.top-0.95
writing to: ./find_genes/EYE_regions.top-0.95.genes.annotated.tsv


__Prepare list of genes for Enrichment analysis in BLAST2GO__ from SNPs with 'ARR >=0.95'

In [30]:
%%bash

cat find_genes/EYE_regions.top-0.95.genes.annotated.tsv | cut -f 4 | grep "gene" -v | sort | uniq \
> find_genes/EYE_regions.top-0.95.genes.txt

wc -l find_genes/EYE_regions.top-0.95.genes.txt

238 find_genes/EYE_regions.top-0.95.genes.txt


##ANALYZE XTX

In [53]:
mkdir XTX

In [54]:
#create the list of Bayenv results files to be processed
import os

bayenv_res_dir = './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates/'
bayenv_files = []

for fil in os.listdir(bayenv_res_dir):
    if fil.endswith(".xtx"):
        print(bayenv_res_dir+"/"+fil)
        bayenv_files.append(bayenv_res_dir+"/"+fil)

./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_4.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_13.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_2.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_12.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_20.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_16.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_6.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_3.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_18.xtx
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_repl

In [56]:
IDs = ['xtx']

In [57]:
print bayenv_files
print "\n%i" %len(bayenv_files)
print IDs
print len(IDs)

['./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_4.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_13.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_2.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_12.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_20.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_16.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_6.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_3.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_18.xtx', './running_bayenv/ANALYSES_FOR_PAPER_exclud

In [58]:
rank_results = QTL.calculate_rank_stats(SNP_map="SNPfiles/full_set.bayenv.SNPmap", infiles = bayenv_files, ids = IDs, prefix = 'XTX/Diplo_xtx_1M')

Total number of SNPs (according to the SNPmap): 11786
Number of Bayenv replicates: 20
Number of environmental factors analysed: 1
parsing bayenv files

processing replicate 0:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_1.xtx

processing replicate 1:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_10.xtx

processing replicate 2:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_11.xtx

processing replicate 3:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_12.xtx

processing replicate 4:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_13.xtx

processing replicate 5:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_14.xtx

processing replicate 6:
./running_bayenv/ANALYSES_FOR_PAPER_exclude_singletons/run_Bayenv/all_replicates//rep_15.xtx

processing replicate 7:

__CREATE POPE PLOTS and extract the SNP ids in the top 5 percent__ (assumes that the script `pope_plot.sh` is in your working directory)

In [59]:
full_rank_files = []

file_dir = 'XTX/'
for id in IDs:
#    print id
    for file in os.listdir(file_dir):
        if file.endswith('_'+id+'.txt'):
#            print [id,file_dir+'/'+file]
            full_rank_files.append([id,file_dir+'/'+file])
            break
        

In [60]:
print full_rank_files

[['xtx', 'XTX//Diplo_xtx_1M_xtx.txt']]


In [61]:
QTL.plot_pope(files_list=full_rank_files, cutoff=0.95, num_replicates=20)

processing xtx:
data in file: XTX//Diplo_xtx_1M_xtx.txt
XTX//Diplo_xtx_1M_xtx.txt
0.95
xtx
XTX//Diplo_xtx_1M_xtx
20
null device 
          1 



In [62]:
QTL.plot_pope(files_list=full_rank_files, cutoff=0.99, num_replicates=20)

processing xtx:
data in file: XTX//Diplo_xtx_1M_xtx.txt
XTX//Diplo_xtx_1M_xtx.txt
0.99
xtx
XTX//Diplo_xtx_1M_xtx
20
null device 
          1 



In [65]:
!cat XTX/Diplo_xtx_1M_xtx-top-0.95.tsv |wc -l
!cat XTX/Diplo_xtx_1M_xtx-top-0.99.tsv |wc -l

544
110


In [ ]:
#calculate p values for xtx
#identifying scaffolds that contain SNPS with ARR >0.95
#only bootstrap SNPs on these scaffolds
IN = open('XTX/Diplo_xtx_1M_xtx-top-0.95.tsv')
scaffolds = []

for line in IN:
    if not line.startswith('chrom'):
            scaffolds.append(line.split("\t")[0])
IN.close()           
scaffolds = sorted(list(set(scaffolds)))

print "%i scaffols to process:\n%s" %(len(scaffolds),str(scaffolds))

QTL.smoothing(dictionary = rank_results, window_size=50000, bootstrap_reps=10000, factor='xtx', 
              to_smooth='avg_rank_rel', verbose=1, chromos=scaffolds)

QTL.write_stats(dictionary=rank_results, 
            out_columns=['std_rank','avg_rank_rel','avg_rank_rel_smoothed','avg_rank_rel_smoothed_p'], 
            factor='xtx', 
            chromos=scaffolds,
            prefix = './XTX/Diplo_xtx_1M-top-0.95-smoothed')

214 scaffols to process:
['scaffold_0', 'scaffold_1', 'scaffold_10', 'scaffold_100', 'scaffold_101', 'scaffold_102', 'scaffold_103', 'scaffold_104', 'scaffold_106', 'scaffold_107', 'scaffold_109', 'scaffold_11', 'scaffold_110', 'scaffold_111', 'scaffold_112', 'scaffold_114', 'scaffold_115', 'scaffold_117', 'scaffold_119', 'scaffold_12', 'scaffold_120', 'scaffold_121', 'scaffold_122', 'scaffold_123', 'scaffold_125', 'scaffold_127', 'scaffold_128', 'scaffold_129', 'scaffold_13', 'scaffold_132', 'scaffold_133', 'scaffold_134', 'scaffold_135', 'scaffold_136', 'scaffold_138', 'scaffold_139', 'scaffold_14', 'scaffold_140', 'scaffold_143', 'scaffold_144', 'scaffold_145', 'scaffold_147', 'scaffold_148', 'scaffold_149', 'scaffold_15', 'scaffold_150', 'scaffold_154', 'scaffold_156', 'scaffold_157', 'scaffold_158', 'scaffold_159', 'scaffold_16', 'scaffold_160', 'scaffold_161', 'scaffold_1616', 'scaffold_162', 'scaffold_163', 'scaffold_164', 'scaffold_167', 'scaffold_168', 'scaffold_17', 'scaffold

Extract IDs of tags with ARR > 0.95 and p < 0.005.

In [5]:
%%bash

cat XTX/Diplo_xtx_1M-top-0.95-smoothed_xtx.txt | \
perl -ne 'chomp; @a=split("\t"); if (($a[4] > 0.95) and ($a[-1] < 0.005)){print "$a[2]\n"}' | \
sort -n | uniq > XTX/XTX.candidates.txt